# Load Package

In [98]:
import pandas as pd
import numpy as np
import os
import networkx as nx
from node2vec import Node2Vec
import matplotlib.pyplot as plt

In [2]:
print (os.getcwd())
os.chdir('D:/OneDrive/ASU/2021 Spring/Applied Project/ASU_Applied_Project_2021/Data')
print (os.getcwd())

C:\Users\Jinhang Jiang
D:\OneDrive\ASU\2021 Spring\Applied Project\ASU_Applied_Project_2021\Data


# Load Data and Explore

In [3]:
data = pd.read_csv("NetworkAnalysis1.csv")

In [4]:
data.head(6)

,Celebrity,Usernames
0,kerwinfrost,vincenthoobyar
1,kerwinfrost,ludwek
2,kerwinfrost,A_G_X10
3,kerwinfrost,CartHard
4,kerwinfrost,jothvm
5,kerwinfrost,saucyboy210


In [30]:
print(*data.Celebrity.unique(),sep="\n")

kerwinfrost
James Bond
beyonce
ZoeSaldana
karlie kloss
Yara Shahidi
naeun
Pharrell Williams
Adriene Mishler
BlackPink
Ninjas Hyper
BadBunny


In [23]:
data.shape

(9764, 2)

In [24]:
print("Number of Celebrities: %0.0f" %len(data.Celebrity.unique()))
print("Number of Users: %0.0f" %len(data.Usernames.unique()))

Number of Celebrities: 12
Number of Users: 1505


In [27]:
print("The percentage of unique values: %0.4f" %(len(data.Usernames.unique())/len(data.Usernames)))

The percentage of unique values: 0.1541


# Generate Adjacency Matrix

In [31]:
#Create matrix
matrix = pd.get_dummies(data.set_index('Usernames')['Celebrity'].astype(str)).max(level=0).sort_index()

In [35]:
matrix.iloc[0:5,0:5]

,Adriene Mishler,BadBunny,BlackPink,James Bond,Ninjas Hyper
Usernames,,,,,
-Fashion-News-,1,1,1,0,1
-SODANK-,0,1,0,0,0
-Sportswear-,1,1,1,0,1
-en-,1,1,1,0,1
12e64m3,1,1,1,0,1


In [38]:
cel_matrix = np.asmatrix(matrix)
cel_matrix_transpose = cel_matrix.transpose()
final_matrix = cel_matrix_transpose.dot(cel_matrix)

network_table = pd.DataFrame(final_matrix)
print(network_table.iloc[0:5,0:5])
print(network_table.shape)

    0    1    2  3    4
0  11   11   11  4   11
1  11  225  166  4   46
2  11  166  166  4  166
3   4    4    4  4    4
4  11   46  166  4   46
(12, 12)


In [39]:
## append index name
Celebrity = list(data.Celebrity.unique())
Celebrity.sort()

network_table.index = Celebrity
network_table.columns = Celebrity
network_table

,Adriene Mishler,BadBunny,BlackPink,James Bond,Ninjas Hyper,Pharrell Williams,Yara Shahidi,ZoeSaldana,beyonce,karlie kloss,kerwinfrost,naeun
Adriene Mishler,11,11,11,4,11,255,36,93,172,186,30,92
BadBunny,11,225,166,4,46,255,36,93,172,186,30,92
BlackPink,11,166,166,4,166,255,36,93,172,186,30,92
James Bond,4,4,4,4,4,4,4,4,4,4,30,4
Ninjas Hyper,11,46,166,4,46,255,36,93,172,186,30,92
Pharrell Williams,255,255,255,4,255,255,36,93,172,186,30,92
Yara Shahidi,36,36,36,4,36,36,36,93,172,186,30,36
ZoeSaldana,93,93,93,4,93,93,93,93,172,93,30,93
beyonce,172,172,172,4,172,172,172,172,172,172,30,172
karlie kloss,186,186,186,4,186,186,186,93,172,186,30,186


In [52]:
#matrix.to_csv('frequency_matrix1.csv')
#network_table.to_csv('network_table1.csv')

# Fit NetworkX

In [57]:
#network_table = pd.read_csv('network_table1.csv', index_col=0)

In [58]:
network_table.head(3)

,Adriene Mishler,BadBunny,BlackPink,James Bond,Ninjas Hyper,Pharrell Williams,Yara Shahidi,ZoeSaldana,beyonce,karlie kloss,kerwinfrost,naeun
Adriene Mishler,11,11,11,4,11,255,36,93,172,186,30,92
BadBunny,11,225,166,4,46,255,36,93,172,186,30,92
BlackPink,11,166,166,4,166,255,36,93,172,186,30,92


In [63]:
#pip install --upgrade networkx

  Found existing installation: networkx 2.3
    Uninstalling networkx-2.3:
      Successfully uninstalled networkx-2.3
Note: you may need to restart the kernel to use updated packages.


In [68]:
graph=nx.from_numpy_matrix(np.matrix(network_table))

In [74]:
node2vec = Node2Vec(graph, dimensions=25, walk_length=10, num_walks=300, workers=4)
model = node2vec.fit(window=10, min_count=1)

Computing transition probabilities: 100%|████████████████████████████████████████████| 12/12 [00:00<00:00, 2005.72it/s]


In [82]:
model.wv.save_word2vec_format('embedding')

In [76]:
vocab, vectors = model.wv.vocab, model.wv.vectors

# get node name and embedding vector index.
name_index = np.array([(v[0], v[1].index) for v in vocab.items()])

# init dataframe using embedding vectors and set index as node name
df =  pd.DataFrame(vectors[name_index[:,1].astype(int)])
df.index = name_index[:, 0]

In [92]:
df = df.sort_index(ascending=True)
df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,-0.147744,-0.044176,0.065672,-0.083068,-0.294284,0.342981,0.184983,-0.237668,0.223541,0.081853,...,-0.161684,0.379380,-0.005959,0.497341,-0.427108,-0.220243,0.023671,-0.278857,-0.674725,0.122246
1,-0.163335,-0.066436,0.052841,-0.114052,-0.319886,0.359776,0.173201,-0.256655,0.205670,0.079700,...,-0.160427,0.385589,0.018726,0.504790,-0.437128,-0.211555,0.047788,-0.275712,-0.660629,0.136803
10,-0.168566,-0.035804,0.098199,-0.129922,-0.360240,0.346373,0.188753,-0.252003,0.193143,0.093038,...,-0.155007,0.396961,0.019826,0.501776,-0.468067,-0.282426,0.058031,-0.310693,-0.749540,0.114207
11,-0.149148,-0.059372,0.066437,-0.085810,-0.300096,0.354894,0.169167,-0.266838,0.240197,0.083486,...,-0.180132,0.362748,-0.004909,0.514344,-0.436735,-0.244049,0.041285,-0.267606,-0.668746,0.117431
2,-0.159323,-0.042151,0.084771,-0.099135,-0.318472,0.362776,0.176587,-0.273824,0.238587,0.097037,...,-0.156165,0.377907,0.022927,0.511601,-0.418118,-0.207591,0.063781,-0.287111,-0.650371,0.134036
3,-0.180591,-0.036817,0.059722,-0.131494,-0.343669,0.364086,0.160926,-0.249468,0.192522,0.103723,...,-0.163323,0.381578,0.017135,0.518914,-0.474209,-0.272068,0.066084,-0.279853,-0.731425,0.090622
4,-0.142373,-0.066504,0.063518,-0.087927,-0.326281,0.340159,0.167958,-0.248896,0.206247,0.102187,...,-0.165456,0.372962,-0.009984,0.506581,-0.414640,-0.239577,0.061644,-0.294533,-0.663465,0.144747
5,-0.137536,-0.075668,0.055149,-0.105118,-0.317359,0.348143,0.163557,-0.249705,0.239935,0.107772,...,-0.161702,0.365878,0.011791,0.501865,-0.418214,-0.233691,0.050897,-0.269465,-0.649101,0.102994
6,-0.120312,-0.063577,0.092431,-0.119435,-0.311107,0.323934,0.184652,-0.245627,0.242171,0.104827,...,-0.135402,0.354035,0.008026,0.503384,-0.439549,-0.232736,0.036261,-0.270565,-0.680053,0.105144
7,-0.147220,-0.047069,0.068712,-0.114014,-0.333816,0.333292,0.183427,-0.263661,0.252079,0.091213,...,-0.145265,0.371257,-0.013080,0.505189,-0.411252,-0.251979,0.031720,-0.286749,-0.681496,0.117579


In [95]:
celebrity_names = ['Adriene Mishler','BadBunny','kerwinfrost','naeun',
 'BlackPink','James Bond','Ninjas Hyper','Pharrell Williams','Yara Shahidi',
 'ZoeSaldana','beyonce','karlie kloss']

In [96]:
df.index = celebrity_names

In [97]:
df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Adriene Mishler,-0.147744,-0.044176,0.065672,-0.083068,-0.294284,0.342981,0.184983,-0.237668,0.223541,0.081853,...,-0.161684,0.379380,-0.005959,0.497341,-0.427108,-0.220243,0.023671,-0.278857,-0.674725,0.122246
BadBunny,-0.163335,-0.066436,0.052841,-0.114052,-0.319886,0.359776,0.173201,-0.256655,0.205670,0.079700,...,-0.160427,0.385589,0.018726,0.504790,-0.437128,-0.211555,0.047788,-0.275712,-0.660629,0.136803
kerwinfrost,-0.168566,-0.035804,0.098199,-0.129922,-0.360240,0.346373,0.188753,-0.252003,0.193143,0.093038,...,-0.155007,0.396961,0.019826,0.501776,-0.468067,-0.282426,0.058031,-0.310693,-0.749540,0.114207
naeun,-0.149148,-0.059372,0.066437,-0.085810,-0.300096,0.354894,0.169167,-0.266838,0.240197,0.083486,...,-0.180132,0.362748,-0.004909,0.514344,-0.436735,-0.244049,0.041285,-0.267606,-0.668746,0.117431
BlackPink,-0.159323,-0.042151,0.084771,-0.099135,-0.318472,0.362776,0.176587,-0.273824,0.238587,0.097037,...,-0.156165,0.377907,0.022927,0.511601,-0.418118,-0.207591,0.063781,-0.287111,-0.650371,0.134036
James Bond,-0.180591,-0.036817,0.059722,-0.131494,-0.343669,0.364086,0.160926,-0.249468,0.192522,0.103723,...,-0.163323,0.381578,0.017135,0.518914,-0.474209,-0.272068,0.066084,-0.279853,-0.731425,0.090622
Ninjas Hyper,-0.142373,-0.066504,0.063518,-0.087927,-0.326281,0.340159,0.167958,-0.248896,0.206247,0.102187,...,-0.165456,0.372962,-0.009984,0.506581,-0.414640,-0.239577,0.061644,-0.294533,-0.663465,0.144747
Pharrell Williams,-0.137536,-0.075668,0.055149,-0.105118,-0.317359,0.348143,0.163557,-0.249705,0.239935,0.107772,...,-0.161702,0.365878,0.011791,0.501865,-0.418214,-0.233691,0.050897,-0.269465,-0.649101,0.102994
Yara Shahidi,-0.120312,-0.063577,0.092431,-0.119435,-0.311107,0.323934,0.184652,-0.245627,0.242171,0.104827,...,-0.135402,0.354035,0.008026,0.503384,-0.439549,-0.232736,0.036261,-0.270565,-0.680053,0.105144
ZoeSaldana,-0.147220,-0.047069,0.068712,-0.114014,-0.333816,0.333292,0.183427,-0.263661,0.252079,0.091213,...,-0.145265,0.371257,-0.013080,0.505189,-0.411252,-0.251979,0.031720,-0.286749,-0.681496,0.117579
